## 빗썸 API 실습

In [ ]:
import os
import hashlib
import math
import time
import base64
import hmac, hashlib
from urllib.parse import urlencode
import requests

### 환경변수 호출

In [ ]:
from dotenv import load_dotenv

load_dotenv()

ACCESS_KEY = os.environ["BITHUMB_API_ACCESS_KEY"]
SECRET_KEY = os.environ["BITHUMB_API_SECRET_KEY"]
SERVER_URL = os.environ["BITHUMB_API_SERVER_URL"]
SERVER_URL

### 토큰 만들기

In [ ]:
def timestamp_millisec():
    mt_string = "%f %d" % math.modf(time.time())
    mt_array = mt_string.split(" ")[:2]
    return mt_array[1] + mt_array[0][2:5]
    
def create_token(endpoint, rgParams):
    """Api-Sign and Api-Nonce information generation.

    # - nonce: it is an arbitrary number that may only be used once.
    # - api_sign: API signature information created in various combinations values.
    """
    endpoint_item_array = {"endpoint": endpoint}

    uri_array = dict(endpoint_item_array, **rgParams)  # Concatenate the two arrays.
    print(f"uri_array {uri_array}")

    str_data = urlencode(uri_array)
    nonce = timestamp_millisec()

    data = endpoint + chr(0) + str_data + chr(0) + nonce
    utf8_data = data.encode("utf-8")

    key = SECRET_KEY
    utf8_key = key.encode("utf-8")

    h = hmac.new(bytes(utf8_key), utf8_data, hashlib.sha512)
    hex_output = h.hexdigest()
    utf8_hex_output = hex_output.encode("utf-8")
    api_sign = base64.b64encode(utf8_hex_output)
    utf8_api_sign = api_sign.decode("utf-8")

    url = SERVER_URL + endpoint
    headers = {
        "Api-Key": ACCESS_KEY,
        "Api-Sign": utf8_api_sign,
        "Api-Nonce": nonce,
        "Content-Type": "application/x-www-form-urlencoded",
    }
    
    return (url, headers, str_data)

In [ ]:
query = {
    "order_currency": "BTC",
    "payment_currency": "KRW",
    "type": "bid",
    "units": str(0.0001),
    "price": str(30000000),
}
create_token("/trade/place", query)

### 주문하기

In [ ]:
def send_order(query):
    print(query)
    info = create_token("/trade/place", query)
    response = requests.post(info[0], headers=info[1], data=info[2])
    return response.json()

### 고정 가격 매수 주문하기

In [ ]:
query = {
    "order_currency": "BTC",
    "payment_currency": "KRW",
    "type": "bid",
    "units": "0.0002",
    "price": "38399000.0",
}
    
send_order(query)

### 고정 가격 매도 주문하기

In [ ]:
query = {
    "order_currency": "BTC",
    "payment_currency": "KRW",
    "type": "bid",
    "units": "0.0002",
    "price": "3820000",
}    
send_order(query)

### 주문 취소하기

In [ ]:
def cancel_order(market, order_id):
    """주문 취소
    Params:
        type, 거래유형 (bid : 매수 ask : 매도), String/필수
        order_id, 매수/매도 주문 등록된 주문번호, String/필수
    """
    query = {
        "order_currency": market,
        "payment_currency": "KRW",
        "order_id": order_id,
    }
    
    info = create_token("/trade/cancel", query)
    response = requests.post(info[0], headers=info[1], data=info[2])
    return response.json()

In [ ]:
cancel_order("BTC", "C0101000000415070670")

### 주문 조회

In [ ]:
def query_order(market, order_id=None):
    query = {"order_currency": market, "payment_currency": "KRW", "order_id": order_id}
    if order_id is None:
        return

    info = create_token("/trade/order_detail", query)
    response = requests.post(info[0], headers=info[1], data=info[2])
    return response.json()

In [ ]:
query_order("BTC", "C0101000000415070670")

In [ ]:
def _usecTime():
    mt = "%f %d" % math.modf(time.time())
    mt_array = mt.split(" ")[:2]
    return mt_array[1] + mt_array[0][2:5]


In [ ]:
_usecTime()

In [ ]:
time.time()

In [ ]:
math.modf(time.time())

In [ ]:
"%f %d" % math.modf(time.time())